In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf


In [2]:
conf = SparkConf()
conf.set("spark.driver.memory", "4g")
conf.set("spark.executor.memory", "4g")
conf.setMaster('local[3]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

In [3]:
import json
import math
import itertools
import random
import sys

In [4]:
reviews_json = sc.textFile('asnlib/publicdata/train_review.json').map(json.loads)

In [5]:
user_business_pairs = reviews_json.map(lambda x: (x.get('user_id'), x.get('business_id'))).distinct()

In [6]:
user_business_pairs.take(10)

[('OLR4DvqFxCKLOEHqfAxpqQ', 'zK7sltLeRRioqYwgLiWUIA'),
 ('0XMLbsJt-fvcQsOHt3_B_Q', 'eTXYID00jGxq1vZpntBUFw'),
 ('bQCHF5rn5lMI9c5kEwCaNA', 'na4Th5DrNauOv-c43QQFvA'),
 ('fiGqQ7pIGKyZ9G0RqWLMpg', 'IVieU4_z2Y9BsRANZYMNNg'),
 ('sDlRFzGHzfpFHaxvJ1JtTg', 'avkOIpslBUFZMQhELpl3JQ'),
 ('qRU7nYMJIV05lpKQq4jVjw', 'jobP3ywRd3QNZ_GCoPG2DQ'),
 ('d976WkRYJVqJyP3vH2Kayg', 'w6-qlTncaWlSxTJwK_HvfQ'),
 ('v5dxjxKLuINf5E6_kYckXg', 'MgSd4P3ATkiywbbraFygHg'),
 ('mLJ3zQwiEzAIBaXb-7iXBw', '3xykzfVY2PbdjKCRDLdzTQ'),
 ('PQ6-FZpUn2oFLyBGvdiG3w', 'bQ_wtZvMb__OhprY5bF9aQ')]

In [7]:
user_tokens_zip = user_business_pairs.map(lambda x: x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x: (x[0], x[1]))

In [8]:
user_tokens_dict = user_tokens_zip.collectAsMap()

In [9]:
business_tokens_zip = user_business_pairs.map(lambda x: x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x: (x[0], x[1]))

In [10]:
business_tokens_dict = business_tokens_zip.collectAsMap()

In [11]:
inverse_business_tokens_dict = {v: k for k, v in business_tokens_dict.items()}

In [12]:
len(business_tokens_dict)

10253

In [13]:
def get_min_hash_functions(num_func, buckets):
    min_hash_func_list = []
    list_a = random.sample(range(1, sys.maxsize - 1), num_func)
    list_b = random.sample(range(0, sys.maxsize - 1), num_func)
    p = 233333333333
    def build_min_hash_func(a,b,p,m):
        def min_hash_func(x):
            return (((a*x + b)%p)%m)
        return min_hash_func

    for a,b in zip(list_a, list_b):
        min_hash_func_list.append(build_min_hash_func(a,b,p,buckets))
        
    return min_hash_func_list

#         min_hash_func_list.append(lambda x: ((a*x + b)%p)%buckets)
#         print (a,b)


In [14]:
min_hash_func_list = get_min_hash_functions(30, len(user_tokens_dict)*2)

In [15]:
user_hashed_values = user_tokens_zip.map(lambda x: (user_tokens_dict.get(x[0]), [min_hash(x[1]) for min_hash in min_hash_func_list]))

In [16]:
user_hashed_values.first()

(0,
 [47926,
  23918,
  7897,
  23406,
  49708,
  846,
  47524,
  46603,
  39776,
  29012,
  20616,
  21993,
  23072,
  19407,
  4288,
  31444,
  36906,
  50223,
  19046,
  3234,
  3016,
  20276,
  3011,
  34423,
  7670,
  31066,
  33515,
  11994,
  12208,
  36285])

In [17]:
user_business_tokenized_map = user_business_pairs.map(lambda x: (user_tokens_dict.get(x[0]), business_tokens_dict.get(x[1]))).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [18]:
user_business_tokenized_map.first()

(7420,
 [7168,
  8897,
  2144,
  1889,
  6853,
  5575,
  6700,
  1327,
  8338,
  7540,
  9941,
  9174,
  7063,
  7259,
  8348,
  253,
  5695])

In [19]:
business_user_tokenized_map = user_business_pairs.map(lambda x: (business_tokens_dict.get(x[1]), user_tokens_dict.get(x[0]))).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [20]:
business_user_tokenized_dict = business_user_tokenized_map.collectAsMap()

In [21]:
business_user_tokenized_dict[6188]

[19874,
 21413,
 24507,
 15211,
 9868,
 2190,
 5212,
 15217,
 5077,
 11514,
 11035,
 5372,
 22429,
 20670]

In [22]:
signature_matrix_rdd = user_business_tokenized_map.leftOuterJoin(user_hashed_values)

In [23]:
signature_matrix_rdd.first()

(7420,
 ([7168,
   8897,
   2144,
   1889,
   6853,
   5575,
   6700,
   1327,
   8338,
   7540,
   9941,
   9174,
   7063,
   7259,
   8348,
   253,
   5695],
  [34671,
   14771,
   33041,
   51469,
   32213,
   31820,
   3615,
   42226,
   39091,
   10717,
   18333,
   36778,
   26352,
   50304,
   27948,
   36184,
   51723,
   13459,
   10546,
   32846,
   8357,
   18013,
   16686,
   20931,
   49318,
   22382,
   34285,
   15623,
   7608,
   44082]))

In [24]:
signature_matrix_rdd = signature_matrix_rdd.map(lambda x: x[1])

In [25]:
signature_matrix_rdd = signature_matrix_rdd.flatMap(lambda business_set: [(x, business_set[1]) for x in business_set[0]]) \

In [26]:
signature_matrix_rdd.filter(lambda x: x[0]==7168).take(3)

[(7168,
  [34671,
   14771,
   33041,
   51469,
   32213,
   31820,
   3615,
   42226,
   39091,
   10717,
   18333,
   36778,
   26352,
   50304,
   27948,
   36184,
   51723,
   13459,
   10546,
   32846,
   8357,
   18013,
   16686,
   20931,
   49318,
   22382,
   34285,
   15623,
   7608,
   44082]),
 (7168,
  [34266,
   44654,
   41291,
   444,
   43717,
   44144,
   27922,
   34744,
   24137,
   6783,
   4136,
   50456,
   48306,
   33730,
   39332,
   39351,
   21223,
   40403,
   45746,
   34809,
   38877,
   38082,
   18981,
   1170,
   15208,
   39702,
   36431,
   9328,
   13253,
   2364]),
 (7168,
  [34606,
   22010,
   15241,
   18190,
   52330,
   49960,
   25769,
   41482,
   42464,
   490,
   25930,
   36683,
   5182,
   29518,
   39633,
   28568,
   28022,
   43404,
   48815,
   42558,
   29848,
   36422,
   13415,
   7544,
   24266,
   4296,
   24061,
   36218,
   4530,
   11226])]

In [27]:
signature_matrix_rdd = signature_matrix_rdd.reduceByKey(lambda a,b: [min(x, y) for x, y in zip(a, b)]).coalesce(2)

In [28]:
len(signature_matrix_rdd.first()[1])

30

In [41]:
candidate_pairs = signature_matrix_rdd \
        .flatMap(lambda x: [(tuple([i, hash(tuple(x[1][i:i+1]))]), x[0]) for i in range(0, 30)]) \
        .groupByKey().map(lambda kv: list(kv[1])).filter(lambda val: len(val) > 1) \
        .flatMap(lambda bid_list: [pair for pair in itertools.combinations(bid_list, 2)])

In [32]:
candidate_pairs = signature_matrix_rdd \
    .flatMap(lambda kv: [(tuple(chunk), kv[0]) for chunk in splitList(kv[1], 30)]) \
    .groupByKey().map(lambda kv: list(kv[1])).filter(lambda val: len(val) > 1) \
    .flatMap(lambda bid_list: [pair for pair in itertools.combinations(bid_list, 2)])


In [34]:
candidate_pairs.count()

3028472

In [42]:
candidate_pairs.count()

3028472

In [29]:
def splitList(value_list, chunk_num):
    """
    split a list in to several chunks
    :param value_list: a list whose shape is [N]
    :param chunk_num: the number of chunk you want to split
    :return: a list of list
    e.g. return [[1,a], [2,b], [3,c], [4,d]] and a + b + c + d = N
    """
    chunk_lists = list()
    size = int(math.ceil(len(value_list) / int(chunk_num)))
    for index, start in enumerate(range(0, len(value_list), size)):
        chunk_lists.append((index, hash(tuple(value_list[start:start + size]))))
    return chunk_lists

def computeJaccard(set1, set2):
    """
    compute Jaccard Similarity
    :param set1:
    :param set2:
    :return: a float number
    """
    return float(float(len(set(set1) & set(set2))) / float(len(set(set1) | set(set2))))

def verifySimilarity(candidate_pairs, index_data_dict,
                     reversed_index_dict, threshold):
    """
    iterate these candidate pairs,
            and compute the jaccard similarity from original data
    :param candidate_pairs: tuple(bidx1, bidx2)
    :param index_data_dict: dict(bidx: [uidx1, uidx2,...])
    :param reversed_index_dict: dict(bidx: bid_str)
    :param threshold: jaccard similarity threshold`
    :return: a list of dict which contain truly similar
            bidx pair and theirs similarity
    """
    result = list()
    temp_set = set()
    for pair in candidate_pairs:
        if pair not in temp_set:
            temp_set.add(pair)
            similarity = computeJaccard(index_data_dict.get(pair[0], set()),
                                        index_data_dict.get(pair[1], set()))
            if similarity >= threshold:
                result.append({"b1": reversed_index_dict[pair[0]],
                               "b2": reversed_index_dict[pair[1]],
                               "sim": similarity})
    return result


In [43]:
result_list = verifySimilarity(candidate_pairs=set(candidate_pairs.collect()),
                               index_data_dict=business_user_tokenized_dict,
                               reversed_index_dict=inverse_business_tokens_dict,
                               threshold=0.05)

In [44]:
len(result_list)

50092

In [37]:
len(candidate_pairs_list)

NameError: name 'candidate_pairs_list' is not defined

In [40]:
50092/59435

0.8428030621687558

In [45]:
5/2

2.5

In [ ]:
candidate_pairs_list = candidate_pairs.collect()

In [ ]:
candidate_pairs.take(10)

In [ ]:
import math
value_list = [698,
   476,
   81,
   350,
   145,
   275,
   580,
   232,
   195,
   730,
   13,
   135,
   62,
   1029,
   955,
   252,
   786,
   97,
   282,
   46,
   0,
   154,
   51,
   313,
   596,
   249,
   231,
   319,
   143,
   464,
   60,
   24,
   156,
   39,
   804,
   1174,
   1067,
   26,
   265,1045,1028,602,301,113,1065,110,714,78,1948,76]
chunk_num = 10
chunk_lists = list()
size = int(math.ceil(len(value_list) / int(chunk_num)))

In [ ]:
size

In [ ]:
for index, start in enumerate(range(0, len(value_list), size)):
    chunk_lists.append((index, hash(tuple(value_list[start:start + size]))))

In [ ]:
chunk_lists

In [ ]:
[tuple([i//5, hash(tuple(value_list[i:i+5]))]) for i in range(0, 50, 5)]

In [ ]:
[list(range(0,50))[i:i + 5] for i in range(0, 50, 5)]